# MobileNetV3 Large

In [ ]:
import os
from pathlib import Path
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, WeightedRandomSampler, Dataset
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from statistics import mode


# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# File Paths

In [ ]:
#Please change these here
training_set_path = '/content/drive/My Drive/Project/train'
validation_set_path = '/content/drive/My Drive/Project/val'
testing_set_path = '/content/drive/My Drive/Project/val'

#In Our second model, we are copying training images to another folder. Please set that training folder here
destination_main_folder = '/content/drive/My Drive/Project/new_train'
destination_side_folder = '/content/drive/My Drive/Project/new_val'
destination_test_folder = '/content/drive/My Drive/Project/new_test'

# Model Setup

In [ ]:
# Setthing up MobilenetV3 Large Model
model = models.mobilenet_v3_large(pretrained=True, progress=True)
model.features[0][0] = nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3,stride=2,padding=1,bias=False)
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, 7)  # 7 classes for skin cancer
model.classifier[2] = nn.Dropout(p=0.4)


# Accessing the block as direct accessing of the convolutional layer was not working
old_dw = model.features[6].block[1][0]  # [6] = block index, [1] = depthwise conv block, [0] = actual Conv2d

# Replace the layer with a dilated version
model.features[6].block[1][0] = nn.Conv2d(
    in_channels=old_dw.in_channels,
    out_channels=old_dw.out_channels,
    kernel_size=5,
    stride=old_dw.stride,
    padding=4,       # padding adjusted for dilation=2
    dilation=2,
    groups=old_dw.groups,
    bias=old_dw.bias
)

for i in range(5):
    for param in model.features[i].parameters():
        param.requires_grad = False

model.to(device)

# Data Augmentations

In [ ]:
# Data augmentation for training
train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.449], std=[0.226]),
])

# Data transform for validation/testing
test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.449], std=[0.226])
])

# Forming the training and validation dataset


In [ ]:
# Paths to train and validation directories
train_path = Path(training_set_path)
valid_path = Path(validation_set_path)

# Combine image files from both directories
train_files = list(train_path.glob('*.jpg'))  # Adjust file extension if needed
valid_files = list(valid_path.glob('*.jpg'))
all_files = train_files + valid_files

# Define indices for splitting
train_idxs = list(range(len(train_files)))
valid_idxs = list(range(len(train_files), len(all_files)))

# Extract label from filename (e.g., "3_xyz.jpg" → 2)
def get_label(file):
    return int(file.name.split('_')[0]) - 1

# Define a custom PyTorch Dataset
class SkinCancerDataset(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        # Load image and label
        img_path = self.file_list[idx]
        label = get_label(img_path)
        img = Image.open(img_path) # Convert to grayscale

        # Apply transformations if any
        if self.transform:
            img = self.transform(img)

        return img, label

# Create datasets
train_dataset = SkinCancerDataset([all_files[i] for i in train_idxs], transform=train_transform)
valid_dataset = SkinCancerDataset([all_files[i] for i in valid_idxs], transform=test_transform)


In [ ]:
# Extract class labels (targets) from the training dataset
targets = [get_label(file) for file in train_files]  # Labels from training dataset files
class_sample_count = np.array([targets.count(t) for t in np.unique(targets)])
print("Class counts:", class_sample_count)

# Compute weights: inverse frequency of each class
weights = 1. / class_sample_count
samples_weight = np.array([weights[t] for t in targets])
samples_weight = torch.from_numpy(samples_weight).float()

# Create a WeightedRandomSampler
sampler = WeightedRandomSampler(samples_weight, num_samples=len(samples_weight), replacement=True)

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=4,pin_memory=True,persistent_workers=True)
val_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4,pin_memory=True,persistent_workers=True)

# Optional: Visualize a sample augmented training image
sample_img, _ = train_dataset[0]
plt.imshow(sample_img.squeeze(), cmap='gray')
plt.title("Augmented Training Image")
plt.show()

# Defining Custom Loss Function

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = nn.CrossEntropyLoss()(inputs, targets)
        pt = torch.exp(-BCE_loss)
        focal_loss = self.alpha * (1 - pt)**self.gamma * BCE_loss
        return focal_loss
criterion = FocalLoss(alpha = 1, gamma = 2)

In [ ]:
from fastai.vision.all import DataLoaders,Learner,accuracy
# Create Fastai DataLoaders
dls = DataLoaders(train_loader, val_loader)
print("Data has been loaded")
# Assuming `dls` is your DataLoaders object, `model` is your model, and `criterion` your loss function
learn = Learner(dls, model, loss_func=criterion, wd=1e-5, metrics=accuracy)
print("Learner Created")
lr_min= learn.lr_find().valley
print(f"Suggested Learning Rates: valley={lr_min}")

In [ ]:
loss_weights = torch.tensor(class_sample_count.sum()/(class_sample_count), dtype=torch.float32,device=device)
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr_min,weight_decay=1e-5)
# scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)
scheduler = CosineAnnealingLR(optimizer, T_max=10)

In [ ]:
num_epochs = 50
train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    batch_iter = 0

    # Training phase
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        batch_iter += 1

        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train
    train_loss_history.append(epoch_loss)
    train_acc_history.append(train_accuracy)

    # Validation phase
    model.eval()
    correct_val = 0
    total_val = 0
    running_val_loss = 0.0
    true_labels = []
    pred_labels = []

    with torch.no_grad():
        for inputs_val, labels_val in val_loader:
            inputs_val = inputs_val.to(device)
            labels_val = labels_val.to(device)

            outputs_val = model(inputs_val)
            val_loss = criterion(outputs_val, labels_val)  # Calculate validation loss
            running_val_loss += val_loss.item()

            _, preds_val = torch.max(outputs_val, 1)
            correct_val += (preds_val == labels_val).sum().item()
            total_val += labels_val.size(0)

            true_labels.extend(labels_val.cpu().numpy())
            pred_labels.extend(preds_val.cpu().numpy())

    # Average validation loss for the epoch
    epoch_val_loss = running_val_loss / len(val_loader)
    val_accuracy = 100 * correct_val / total_val
    val_loss_history.append(epoch_val_loss)
    val_acc_history.append(val_accuracy)


    # Update the scheduler based on validation loss
    scheduler.step()

    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch+1}: Learning Rate = {current_lr}")
    # Print metrics for the epoch
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.4f} | Train Acc: {train_accuracy:.2f}% | Val Loss: {epoch_val_loss:.4f} | Val Acc: {val_accuracy:.2f}%")
    print(classification_report(true_labels, pred_labels, target_names=['0','1','2','3','4','5','6'], zero_division=0))
    print("---------------------------------------------------")

In [ ]:
# Plot training loss and accuracy
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs+1), train_loss_history, marker='*')
plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.title("Training Loss per Epoch")

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs+1), train_acc_history, marker='*', label='Train Acc')
plt.plot(range(1, num_epochs+1), val_acc_history, marker='*', label='Val Acc')
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.show()

# Generate the confusion matrix
cm = confusion_matrix(true_labels, pred_labels)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Reds", xticklabels=['0','1','2','3','4','5','6'], yticklabels=['0','1','2','3','4','5','6'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# PLease change to testing_set_path, have set it to validation so that it runs
test_path = Path(validation_set_path)

# Combine image files from both directories
test_files = list(test_path.glob('*.jpg'))

# Define indices for splitting
test_idxs = list(range(len(test_files)))

# Create datasets
test_dataset = SkinCancerDataset([all_files[i] for i in test_idxs], transform=test_transform)

# Create DataLoaders
batch_size = 32
testing = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4,pin_memory=True,persistent_workers=True)

correct = 0
total = 0
model.eval()
true_labels = []
pred_labels = []
class_correct = [0] * 7  # Initialize list to store correct predictions for each class
class_total = [0] * 7   # Initialize list to store total samples for each class

with torch.no_grad():
    for inputs, labels in testing:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(predicted.cpu().numpy())

        # Calculate per-class accuracy
        for i in range(len(labels)):
            label = labels[i].item()
            class_total[label] += 1
            class_correct[label] += (predicted[i] == label).item()

print(f'Overall Accuracy: {100 * correct / total}%')

# Print per-class accuracy
for i in range(7):
    accuracy = 100 * class_correct[i] / class_total[i] if class_total[i] else 0  # Handle zero division
    print(f'Accuracy of class {i}: {accuracy:.2f}%')


# **Model 2: MobileNetV2**

In [ ]:
import shutil

# Path to trianing set
source_folder = training_set_path

#Destination For New folder

# Make sure the destination main folder exists
os.makedirs(destination_main_folder, exist_ok=True)

# Loop through each file in the source folder
for filename in os.listdir(source_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):  # Add more extensions if needed
        try:
            # Extract class number (before underscore)
            class_num = filename.split('_')[0]
            class_folder_name = f'class{class_num}'

            # Create the class subfolder inside the new main folder
            class_folder_path = os.path.join(destination_main_folder, class_folder_name)
            os.makedirs(class_folder_path, exist_ok=True)

            # Copy the file
            src_path = os.path.join(source_folder, filename)
            dst_path = os.path.join(class_folder_path, filename)
            shutil.copy2(src_path, dst_path)  # copy2 preserves metadata

            print(f'Copied {filename} to {class_folder_name}')
        except Exception as e:
            print(f'Error processing {filename}: {e}')


In [ ]:
# Path to trianing set
source_folder = validation_set_path

#Destination For New folder

# Make sure the destination main folder exists
os.makedirs(destination_side_folder, exist_ok=True)

# Loop through each file in the source folder
for filename in os.listdir(source_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):  # Add more extensions if needed
        try:
            # Extract class number (before underscore)
            class_num = filename.split('_')[0]
            class_folder_name = f'class{class_num}'

            # Create the class subfolder inside the new main folder
            class_folder_path = os.path.join(destination_side_folder, class_folder_name)
            os.makedirs(class_folder_path, exist_ok=True)

            # Copy the file
            src_path = os.path.join(source_folder, filename)
            dst_path = os.path.join(class_folder_path, filename)
            shutil.copy2(src_path, dst_path)  # copy2 preserves metadata

            print(f'Copied {filename} to {class_folder_name}')
        except Exception as e:
            print(f'Error processing {filename}: {e}')


In [ ]:
import cv2
import random
from glob import glob
from tqdm import tqdm
from albumentations import (
    Compose, HorizontalFlip, Rotate, RandomBrightnessContrast,
    ShiftScaleRotate, GaussianBlur, RandomShadow, RGBShift
)
from albumentations.pytorch import ToTensorV2
import numpy as np

source_dir = destination_main_folder
target_count = 2000                # Target samples per class
placeholder = "_synthetic"        # To tag synthetic images so we can delete these later

# Data Augmentations for synthetic images
augment = Compose([
    HorizontalFlip(p=0.5),
    Rotate(limit=30, p=0.5),
    RandomBrightnessContrast(p=0.5),
    ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=20, p=0.5),
    GaussianBlur(p=0.3),
    RGBShift(p=0.3),
    RandomShadow(p=0.3)
])

#creating synthetic images
for class_dir in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_dir)
    if not os.path.isdir(class_path):
        continue

    images = glob(os.path.join(class_path, "*.jpg")) + \
             glob(os.path.join(class_path, "*.png")) + \
             glob(os.path.join(class_path, "*.jpeg"))

    current_count = len(images)
    print(f"Class {class_dir}: {current_count} images")

    if current_count >= target_count:
        continue
    needed = target_count - current_count
    print(f"→ Generating {needed} synthetic images for class {class_dir}")

    for i in tqdm(range(needed)):
        img_path = random.choice(images)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        augmented = augment(image=image)['image']
        augmented = cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR)

        base_name = os.path.basename(img_path)
        name, ext = os.path.splitext(base_name)
        new_name = f"{name}_{placeholder}_{i}{ext}"
        new_path = os.path.join(class_path, new_name)

        cv2.imwrite(new_path, augmented)

print("\nSynthetic image generation complete.")


In [ ]:
import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay


# Data augmentation and preprocessing
train_transforms = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.RandomRotation(45),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=30, translate=(0.1, 0.1), scale=(0.8, 1.2)),
    transforms.GaussianBlur(kernel_size=(5, 5)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalization for ImageNet pre-trained models
])

val_transforms = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Paths to training and validation datasets
train_dataset = ImageFolder(root = destination_main_folder, transform = train_transforms)
val_dataset = ImageFolder(root = destination_side_folder, transform = val_transforms)

In [ ]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Calculate class counts for weighted sampling
targets = [s[1] for s in train_dataset.samples]
class_sample_count = np.array([targets.count(t) for t in np.unique(targets)])
print("Class counts:", class_sample_count)

# Compute weights: inverse frequency
weights = 1. / class_sample_count
samples_weight = np.array([weights[t] for t in targets])
samples_weight = torch.from_numpy(samples_weight).float()
sampler = WeightedRandomSampler(samples_weight, num_samples=len(samples_weight), replacement=True)

In [ ]:
# Load datasets
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler, num_workers=4, persistent_workers=True,  pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle = False, num_workers=4, persistent_workers=True, pin_memory=True)

In [ ]:
# Load pre-trained MobileNetV2 model and modify it
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, len(train_dataset.classes))
model = model.to(device)

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return focal_loss.mean()

# Loss function and optimizer
criterion = FocalLoss(alpha=0.25, gamma=2)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
# Training function
def train_model_with_accuracy(model, criterion, optimizer, scheduler, num_epochs=20):
    train_loss_history = []
    val_loss_history = []
    train_acc_history = []
    val_acc_history = []

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')

        # Training phase
        model.train()
        running_loss = 0.0
        correct_train = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_train += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = correct_train.double() / len(train_dataset)
        train_loss_history.append(epoch_loss)
        train_acc_history.append(epoch_acc.item())

        print(f'Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_acc:.4f}')

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct_val = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                correct_val += torch.sum(preds == labels.data)

        val_epoch_loss = val_loss / len(val_dataset)
        val_epoch_acc = correct_val.double() / len(val_dataset)
        val_loss_history.append(val_epoch_loss)  # Append validation loss to the list
        val_acc_history.append(val_epoch_acc.item())

        print(f'Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_epoch_acc:.4f}')

        scheduler.step()

    return train_loss_history, val_loss_history, train_acc_history, val_acc_history  # Return loss histories

In [ ]:
train_loss_history, val_loss_history, train_acc_history, val_acc_history = train_model_with_accuracy(model, criterion, optimizer, scheduler)

In [ ]:
# Plot training and validation accuracy curves
plt.figure(figsize=(10, 5))
plt.plot(train_acc_history, label='Train Accuracy')
plt.plot(val_acc_history, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
# Plot the loss curves
plt.figure(figsize=(10, 5))
plt.plot(train_loss_history, label='Training Loss')
plt.plot(val_loss_history, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Curves')
plt.legend()
plt.show()

In [ ]:
def plot_confusion_matrix(model, data_loader, dataset_type ):
  model.eval()
  all_preds = []
  all_labels = []

  with torch.no_grad():
      for inputs, labels in data_loader:
          inputs, labels = inputs.to(device), labels.to(device)
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          all_preds.extend(preds.cpu().numpy())
          all_labels.extend(labels.cpu().numpy())

  # Calculate confusion matrix
  cm = confusion_matrix(all_labels, all_preds)

  # Plot the confusion matrix using seaborn
  plt.figure(figsize=(10, 8))
  sns.heatmap(cm, annot=True, fmt="d", cmap="Blues") #added cmap value
  plt.title(f"Confusion Matrix for {dataset_type} Set")


  # Calculate per-class accuracy
  class_accuracy = cm.diagonal() / cm.sum(axis=1)

  # Print per-class accuracy
  print(f"--- Class Accuracy for {dataset_type} Set ---")  # Added distinction
  for i, accuracy in enumerate(class_accuracy):
      print(f"Accuracy for class {train_dataset.classes[i]}: {accuracy*100:.4f}")

  plt.tight_layout()
  plt.show()  # Display the plot immediately to separate

In [ ]:
plot_confusion_matrix(model, train_loader, dataset_type="Training")  # Confusion matrix for training set
plot_confusion_matrix(model, val_loader, dataset_type="Validation")  # Confusion matrix for validation set

In [ ]:
print(f"Final Validation Accuracy: {val_acc_history[-1]:.4f}")

In [ ]:
try:
    shutil.rmtree(destination_main_folder)
    print(f'Deleted folder: {destination_main_folder}')
except Exception as e:
    print(f'Error deleting folder {destination_main_folder}: {e}')

In [ ]:
try:
    shutil.rmtree(destination_side_folder)
    print(f'Deleted folder: {destination_side_folder}')
except Exception as e:
    print(f'Error deleting folder {destination_side_folder}: {e}')

## **Test Block:**

In [ ]:
# Path to trianing set
source_folder = testing_set_path

#Destination For New folder

# Make sure the destination main folder exists
os.makedirs(destination_test_folder, exist_ok=True)

# Loop through each file in the source folder
for filename in os.listdir(source_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):  # Add more extensions if needed
        try:
            # Extract class number (before underscore)
            class_num = filename.split('_')[0]
            class_folder_name = f'class{class_num}'

            # Create the class subfolder inside the new main folder
            class_folder_path = os.path.join(destination_test_folder, class_folder_name)
            os.makedirs(class_folder_path, exist_ok=True)

            # Copy the file
            src_path = os.path.join(source_folder, filename)
            dst_path = os.path.join(class_folder_path, filename)
            shutil.copy2(src_path, dst_path)  # copy2 preserves metadata

            print(f'Copied {filename} to {class_folder_name}')
        except Exception as e:
            print(f'Error processing {filename}: {e}')


In [ ]:
new_test_dataset = ImageFolder(root = destination_test_folder, transform = val_transforms)
new_test_loader = DataLoader(new_test_dataset, batch_size=32, shuffle = False, num_workers=4, persistent_workers=True, pin_memory=True)
correct = 0
total = 0
model.eval()
true_labels = []
pred_labels = []
class_correct = [0] * 7  # Initialize list to store correct predictions for each class
class_total = [0] * 7   # Initialize list to store total samples for each class

with torch.no_grad():
    for inputs, labels in new_test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(predicted.cpu().numpy())

        # Calculate per-class accuracy
        for i in range(len(labels)):
            label = labels[i].item()
            class_total[label] += 1
            class_correct[label] += (predicted[i] == label).item()

print(f'Overall Accuracy: {100 * correct / total}%')

# Print per-class accuracy
for i in range(7):
    accuracy = 100 * class_correct[i] / class_total[i] if class_total[i] else 0  # Handle zero division
    print(f'Accuracy of class {i}: {accuracy:.2f}%')